Version: 2019.12.12

---

# Intelligent Systems - Decision Tree Learning

# Aufgabe 1 - Information Gain

* Rufen Sie sich auch den Pseudocode für das Anlernen eines Decision Tree Models ins Gedächtnist:
![DTL](https://docs.google.com/uc?id=1U6sPhZkVc0VAykERjN0bJ3ImWU-zNtge)

## Aufgabe

Betrachten Sie ein Datenset aus 400 Beispiele der Klasse C1 und 400 Beispiele der Klasse C2.
Nehmen Sie an, ein Decision Tree Model A teilt diese Beispiele in (300,100) für den ersten Blattknoten und (100,300) für den zweiten Blattknoten, wobei (n,m) bedeutet, dass n Beispiele zu C1 gehören und m Beispiele zu C2.
Genau so teilt ein Decision Tree Model B die Beispiele in (200,400) und (200,0).

**(1)** Berechnen Sie die Misclassification Rate für beide DTMs und zeigen Sie, dass diese gleich sind!*

**(2)** Berechnen Sie den Information Gain für beide DTMs und zeigen Sie, dass der Information Gain von DTM $B$ größer ist als von $A$!*

**(3)** Vervollständigen Sie den Code zur Berechnung der Misclassification Rate und  des Information Gains!*



In [2]:
from math import log2
# rest entropy and information gain for A
ha = -(400/800*log2(400/800)) - (400/800*log2(400/800))
ra = (400/800)*(-(300/400)*log2(300/400)-(1/4)*log2(1/4)) + (1/2)*(-(1/4)*log2(1/4)-(3/4)*log2(3/4))
iga = ha - ra

print("H for A: ", ha)
print("Remaining Entropy R for tree A: ", ra)
print("Information Gain for tree A: ", iga)

H for A:  1.0
Remaining Entropy R for tree A:  0.8112781244591328
Information Gain for tree A:  0.18872187554086717


In [3]:
# rest entropy and information gain for B
hb = - 400/800*log2(400/800) - 400/800*log2(400/800)
rb = 1/4*(-1*log2(1)-0) + 3/4*(-(1/3)*log2(1/3)-(2/3)*log2(2/3))
igb = hb - rb
print("H for B: ", hb)
print("Remaining Entropy R for tree B", rb)
print("Information Gain for tree B: ", igb)

H for B:  1.0
Remaining Entropy R for tree B 0.6887218755408672
Information Gain for tree B:  0.31127812445913283


**(4)** Gegeben seien zwei binäre Variablen $A$ und $B$, mit denen ein Decision Tree zur Klassifikation trainiert wird. Dieser verwendet die XOR-Funktion. Die Trainingsdaten bestehen aus Beispielen für alle möglichen Kombinationen von Attributen und Klassen. Anschließend werden ungelabelte (Klasse nicht bekannt) Daten mit allen möglichen Werten für $A$ und $B$ mit dem trainierten Decision Tree klassifiziert. Die klaissifizierten Daten werden wiederrum für das Training eines neuen Decision Trees genutzt. Erhalten wir den gleichen Decision Tree?*

# Aufgabe 2 - Recursive Split Algorithm

![DTL](https://docs.google.com/uc?id=1U6sPhZkVc0VAykERjN0bJ3ImWU-zNtge)

Auf der Basis der Daten der amerikanischen Volkszählung wollen wir die folgenden zwei Klassen bestimmen.

    >50K$, <=50K$

 **Die Attribute sind**: 

* **age**: continuous.  
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.  
* **fnlwgt**: continuous. (= final weight)
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* **education-num**:  continuous. 
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.  
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.  
* **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
(Comment: this term is (was?) quite common in the american litrature, although ...  search the net for the debate in the U.S. and elsewhere.) 
* **sex**: Female, Male.  
* **capital-gain**: continuous.
* **capital-loss**: continuous.
* **capital-loss**: continuous.
* **capital-loss**: continuous.
* **hours-per-week**: continuous.
* **native-country**:  United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Wir benutzen keine kontinuierlichen Attribute. 

Daten finden Sie auf der Website (adult.txt ...)

## Aufgabe: Vervollständigen Sie den unteren Code
 **Es gibt Blanks in den folgenden Funktionen**: 
- calc_entropy()
- calc_ig()
- majority()
- choose_best_attr()
- dtree_learning()
- dtree_classify()
- dtree_test()

In [ ]:
""" Intelligent Systems TUD 2018, Ex.1

Decision Tree Learning:
Based on american census data you want to predict two classes of income of people:
>50K$, <=50K$.

We do not use continuous attributes for this first decision tree task.
"""
import numpy as np

__author__ = 'Benjamin Guthier'

from math import log


def openfile(path, fname):
    """opens the file at path+fname and returns a list of examples and attribute values.
    examples are returned as a list with one entry per example. Each entry then
    is a list of attribute values, one of them being the class label. The returned list attr
    contains one entry per attribute. Each entry is a list of possible values or an empty list
    for numeric attributes.
    """
    datafile = open(path + fname, "r")
    examples = []
    for line in datafile:
        line = line.strip()
        line = line.strip('.')
        # ignore empty lines. comments are marked with a |
        if len(line) == 0 or line[0] == '|':
            continue
        ex = [x.strip() for x in line.split(",")]
        examples.append(ex)

    attr = []
    for i in range(len(examples[0])):
        values = list({x[i] for x in examples})  # set of all different attribute values
        if values[0].isdigit():  # if the first value is a digit, assume all are numeric
            attr.append([])
        else:
            attr.append(values)

    return examples, attr


def calc_entropy(examples, cls_index):
    """calculates the entropy over all examples. The index of the class label in the example
    is given by cls_index. Can also be the index to an attribute.
    """
    global attr
    # get attributes of examples with index cls_index
    example_classifications = [example[___] for example in examples]
    # get unique counts of example classifications
    unique, counts = np.unique(___, return_counts=True)

    # normalize counts to total number of examples for getting probs
    probs = counts/len(examples)
    # print(probs)

    entropy = ___

    return entropy


def calc_ig(examples, attr_index, cls_index):
    """Calculates the information gain over all examples for a specific attribute. The
    class index must be specified.

    uses calc_entropy
    """
    global attr
    # get attributes of examples with index attr_index
    example_attributes = [example[___] for example in examples]

    # get unique counts of example_attributes
    unique, counts = np.unique(example_attributes, return_counts=True)

    # example split
    remainder = 0
    for j in range(len(unique)):
        # get all examples with attribute unique[j]
        examples_unique = [example for example in examples if example[attr_index] == unique[j]]
        remainder += len(___)/len(examples)*calc_entropy(___, cls_index)
    return calc_entropy(examples, cls_index) - remainder


def majority(examples, attr_index):
    """Returns the value of attribute "attr_index" that occurs the most often in the examples."""
    # create a flat list of all attribute values (with duplicates, so we can count)
    attr_vals = [example[___] for example in examples]

    # among all unique attribute values, find the maximum with regards to occurrence in the attr_vals list
    maj_attr_val = ___
    return maj_attr_val


def choose_best_attr(examples, attr_avail, cls_index):
    """Iterates over all available attributes, calculates their information gain and returns the one
    that achieves the highest. attr_avail is a list of booleans corresponding to the list of attributes.
    it is true if the attribute has not been used in the tree yet (and is not numeric).
    """
    igs = []  # list of information gains for each attribute

    for j in range(len(attr)):
        if attr_avail[j]:
            igs.append(___)
        else:
            igs.append(-1)

    return igs.index(max(igs))  # return index of the attribute with highest IG


def dtree_learning(examples, attr_avail, default, cls_index):
    """Implementation of the decition tree learning algorithm according to the pseudo code
    in the lecture. Receives the remaining examples, the remaining attributes (as boolean list),
    the default label and the index of the class label in the attribute vector.
    Returns the root node of the decision tree. Each tree node is a tuple where the first entry is
    the index of the attribute that has been used for the split. It is "None" for leaf nodes.
    The second entry is a list of subtrees of the same format. The subtrees are ordered in the
    same way as the attribute values in "attr". For leaf nodes, the second entry is the predicted class.

    uses choose_best_attr, majority, dtree_learning
    """
    global attr
    if len(examples) == 0:
        # is leaf
        # no examples left -> majority class prediction of parent (default)
        return [None, default]
    elif len({x[cls_index] for x in examples}) == 1:
        # is leaf
        # uniform classes
        return [___, ___]
    elif attr_avail.count(True) == 0:
        # is leaf
        # no attribute left to split -> majority class prediction
        return [___, ___]
    else:
        best = choose_best_attr(examples, attr_avail, cls_index)
        tree = [best, []]
        # append subtrees for each attribute value of 'best'
        for v in attr[best]:
            examples_v = [x for x in examples if x[___] == v]
            new_attr_avail = attr_avail.copy()
            new_attr_avail[best] = False
            subtree = dtree_learning(___, ___, ___, cls_index)
            tree[1].append(subtree)
        return tree


def dtree_classify(dtree, x):
    """Classifies a single example x using the given decision tree. Returns the predicted class label.
    """
    # attribute index of splitting attribute
    attr_split_index = dtree[___]

    if attr_split_index is not ___:
        # get attribute value for example
        example_split_attr = x[attr_split_index]

        # subtree position
        subtree_pos = attr[attr_split_index].index(example_split_attr)

        return dtree_classify(dtree[1][subtree_pos], x)  # descend into subtree recursively
    else:
      # leaf, return class prediction
        return dtree[1]


def dtree_test(dtree, examples, cls_index):
    """Classify all examples using the given decision tree. Prints the achieved accuracy."""
    correct = 0
    for j in range(len(examples)):
        # check if predicted class label for example[j] is the correct one
        if dtree_classify(dtree, ___) == ___:
            correct += 1

    print("{} out of {} correct ({:.2f}%)".format(correct, len(examples), correct / len(examples) * 100))


path = "./"  # directory of your data
datafile = "adult.data.txt"
testfile = "adult.test.txt"
examples, attr = openfile(path, datafile)  # load the training set
test, test_attr = openfile(path, testfile)  # load the test set
cls_index = len(attr) - 1  # the last attribute is assumed to be the class label
# attr_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "class"]

attr_avail = []  # marks which attributes are available for splitting (not numeric and not the class label)
for i in range(len(attr)):
    # the list attr[i] contains all possible values of attribute i. It is empty for numeric attributes.
    attr_avail.append(len(attr[i]) > 0 and i != cls_index)

# print(attr_avail)
dtree = dtree_learning(examples, attr_avail, [], cls_index)

print("Before removal of unknowns: ")
print("Training Set")
dtree_test(dtree, examples, cls_index)
print("Test Set")
dtree_test(dtree, test, cls_index)

# Extra task removal of unknowns
examples_removed = []
test_removed = []

for example in examples:
    # print(example.__contains__("?"))
    if not example.__contains__("?"):
        examples_removed.append(example)

for example in test:
    if not example.__contains__("?"):
        test_removed.append(example)

# print(len(examples_removed))
# print(len(test_removed))
dtree = dtree_learning(examples_removed, attr_avail, [], cls_index)

print("After removal of unknowns: ")
print("Training Set")
dtree_test(dtree, examples_removed, cls_index)
print("Test Set")
dtree_test(dtree, test_removed, cls_index)